# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

R. Burn  ->  R. Burn  |  ['R. Burn']
J. Shi  ->  J. Shi  |  ['J. Shi']
G. Chauvin  ->  G. Chauvin  |  ['G. Chauvin']
M. Häberle  ->  M. Häberle  |  ['M. Häberle']


H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
S. Kraus  ->  S. Kraus  |  ['S. Kraus']
Arxiv has 66 new papers today
          7 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/7 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2509.09762


extracting tarball to tmp_2509.09762... done.


R. Burn  ->  R. Burn  |  ['R. Burn']


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure bias_harps
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure bias_harps as tmp_2509.09762/./bias_harps.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))


...

extracting tarball to tmp_2509.10440...

 done.


Found 171 bibliographic references in tmp_2509.10440/main.bbl.
Issues with the citations
syntax error in line 221: '=' expected
Retrieving document from  https://arxiv.org/e-print/2509.10451


extracting tarball to tmp_2509.10451...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.09762-b31b1b.svg)](https://arxiv.org/abs/2509.09762) | **The New Generation Planetary Population Synthesis (NGPPS). VII. Statistical comparison with the HARPS/Coralie survey**  |
|| A. Emsenhuber, et al. -- incl., <mark>R. Burn</mark> |
|*Appeared on*| *2025-09-15*|
|*Comments*| *Accepted for publication in A&A. Abstract abridged to meet arXiv requirements*|
|**Abstract**|            We seek to quantify the fidelity with which modern population syntheses reproduce observations in view of their use as predictive tools. We compared synthetic populations from the Generation 3 Bern Model of Planet Formation and Evolution (core accretion, solar-type host stars) and the HARPS/Coralie radial velocity sample. We biased the synthetic planet population according to the completeness of the observed data and performed quantitative statistical comparisons and systematically identified agreements and differences. Our nominal population reproduces many of the main features of the HARPS planets: two main groups of planets (close-in sub-Neptunes and distant giants), a bimodal mass function with a less populated `desert', an observed mean multiplicity of about 1.6, and several key correlations. The remaining discrepancies point to areas that are not fully captured in the model. For instance, we find that the synthetic population has 1) in absolute terms too many planets by ~70%, 2) a `desert' that is too deep by ~60%, 3) a relative excess of giant planets by ~40%, 4) planet eccentricities that are on average too low by a factor of about two (median of 0.07 versus 0.15), and 5) a metallicity effect that is too weak. Finally, the synthetic planets are overall too close to the star compared to the HARPS sample. The differences allowed us to find model parameters that better reproduce the observed planet masses, for which we computed additional synthetic populations. We find that physical processes appear to be missing and that planets may originate on wider orbits than our model predicts. Mechanisms leading to higher eccentricities and slower disc-limited gas accretion also seem necessary. We advocate that theoretical models should make a quantitative comparison between the many current and future large surveys to better understand the origins of planetary systems. (Abridged.)         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.09878-b31b1b.svg)](https://arxiv.org/abs/2509.09878) | **Stellar halo subtraction alternative for accreting companions' characterization with integral field spectroscopy: Analytical and on-sky demonstration on the PDS70, HTLup, and YSES1 systems**  |
|| R. Julo, et al. -- incl., <mark>G. Chauvin</mark> |
|*Appeared on*| *2025-09-15*|
|*Comments*| *14 pages, 15 figures, to be published in A&A*|
|**Abstract**|            Context. Medium-resolution IFS, such as MUSE at the VLT, are equipped to detect the emission lines of faint accreting companions when associated with dedicated stellar halo subtraction methods. We recently proposed a new approach based on polynomial modulations of a stellar spectrum estimate across the field of view, with orthogonal polynomials and lines masking. Aims. We seek to highlight and quantify analytically and on real data the benefits of this new approach over the one classically used, particularly with regard to distortions of the extracted spectra. Methods. We carried out analytical calculations based on simple toy models. Simulations of the most extreme situations identified were used to highlight these problems and corrections. Archival VLT/MUSE data of the young PDS70 and HTLup systems were used to vet the detection and characterization capabilities using on-sky observations. New images of the YSES1 planetary system were used to further illustrate the gains. Results. We show that the state-of-the-art method, based on low-pass filtering, can lead to the self-subtraction of the emission lines and modify the neighboring continua. We show that the proposed technique corrects these characterization problems, while maintaining the same detection capabilities. The two protoplanets PDS70 b and c were detected with 5sigma significance. The Halpha line estimate of the HTLup B stellar companion was improved by ~30% for the integrated flux and by ~8% for the 10%-width. As for YSES1 b, we found it uniquely displays a combination of Halpha, Hbeta, CaII H&K triplet, and HeI lines in emission. Conclusions. The proposed method better preserves the spectral information, notably the emission line fluxes and profiles, while achieving similar detection performance. Based on a linear and parametric approach, it can be extended and/or combined with additional faint signal search algorithms.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.10144-b31b1b.svg)](https://arxiv.org/abs/2509.10144) | **Cluster Ages to Reconstruct the Milky Way Assembly (CARMA) IV. Chrono-dynamics of seven old star clusters in the Large Magellanic Cloud and the peculiar origin of NGC 1841**  |
|| F. Niederhofer, et al. -- incl., <mark>M. Häberle</mark> |
|*Appeared on*| *2025-09-15*|
|*Comments*| *Accepted for publication in A&A, 22 pages, 13 figures, 4 tables*|
|**Abstract**|            In this study, we report conclusive evidence for an ancient star cluster that has been accreted by the Large Magellanic Cloud (LMC). By leveraging observations from the Hubble Space Telescope (HST), we investigate the chrono-dynamical structure of a sample of seven old star clusters within the LMC in a self-consistent way. The multi-epoch nature of the dataset allowed the determination of high-precision proper motions for the clusters. Employing an isochrone-fitting methodology, we additionally infer from the deep high-resolution HST data homogeneous and robust estimates for their distances, ages and metallicities. Supplementing these data with literature line-of-sight velocities, we investigate the full 3-dimensional dynamics of the clusters within the frame of the LMC. With respect to the other clusters in our sample, NGC 1841 depicts a peculiar case. Its position in the age-metallicity plane, that makes it about 1 Gyr younger than the other metal-poor LMC clusters, but also its dynamical properties with a radial orbit almost perpendicular to the LMC disc plane, clearly advocates for a different origin. We thus conclude that NGC 1841 has likely been accreted by the LMC from a smaller galaxy. The other clusters in our sample show disc-like kinematics, with the case of NGC 2210 being peculiar, based on its inclined orbit. Their coherent age-metallicity relation closely resembles that of Gaia-Sausage-Enceladus globular clusters, thus suggesting a similar early evolution for the two dwarf galaxies. We do not find clear-cut chrono-kinematic evidence that NGC 2005 has been accreted by the LMC as suggested by a previous study based on its chemical abundance pattern. Regardless of its nature, its very old age illustrates that peculiar chemical evolutions already emerge at very early times.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.10387-b31b1b.svg)](https://arxiv.org/abs/2509.10387) | **The large-scale kinematics of young stars in the Milky Way disc: first results from SDSS-V**  |
|| E. Zari, et al. -- incl., <mark>H.-W. Rix</mark> |
|*Appeared on*| *2025-09-15*|
|*Comments*| *13 pages (13 figures) + 3 pages of appendices (5 figures). Under review at A&A; revised after first referee comments*|
|**Abstract**|            We present a first large-scale kinematic map of $\sim$50,000 young OB stars ($T_{\rm eff} \geq 10,000$ K), based on BOSS spectroscopy from the Milky Way Mapper OB program in the ongoing Sloan Digital Sky Survey V (SDSS-V). Using photogeometric distances, line-of-sight velocities and Gaia DR3 proper motions, we map 3D Galactocentric velocities across the Galactic plane to $\sim$5 kpc from the Sun, with a focus on radial motions ($v_R$). Our results reveal mean radial motion with amplitudes of $\pm 30$ km/s that are coherent on kiloparsec scales, alternating between inward and outward motions. These $\bar{v}_R$ amplitudes are considerably higher than those observed for older, red giant populations. These kinematic patterns show only a weak correlation with spiral arm over-densities. Age estimates, derived from MIST isochrones, indicate that 85% of the sample is younger than $\sim300$ Myr and that the youngest stars ($\le 30$ Myr) align well with density enhancements. The age-dependent $\bar{v}_R$ in Auriga makes it plausible that younger stars exhibits different velocity variations than older giants. The origin of the radial velocity features remains uncertain, and may result from a combination of factors, including spiral arm dynamics, the Galactic bar, resonant interactions, or phase mixing following a perturbation. The present analysis is based on approximately one-third of the full target sample. The completed survey will enable a more comprehensive investigation of these features and a detailed dynamical interpretation.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.10440-b31b1b.svg)](https://arxiv.org/abs/2509.10440) | **Euclid: Early Release Observations -- The star cluster systems of the Local Group dwarf galaxies IC 10 and NGC 6822**  |
|| J. M. Howell, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-09-15*|
|*Comments*| **|
|**Abstract**|            Star clusters are valuable indicators of galaxy evolution, offering insights into the buildup of stellar populations across cosmic time. Understanding intrinsic star cluster populations of dwarf galaxies is particularly important given their role in the hierarchical growth of larger systems. Using Euclid Early Release Observation data, we study star clusters in two star-forming dwarf irregulars in the Local Group, NGC 6822 and IC 10 [$M_\star \sim$ (1--4) $\times10^8 M_\odot$]. With Euclid, clusters are resolved into individual stars across the main bodies and haloes of both galaxies. Visual inspection of $I_E$ images uncovers 30 new cluster candidates in NGC 6822 and 16 in IC 10, from compact to extended clusters. We re-evaluate literature candidates, producing combined catalogues of 52 (NGC 6822) and 71 (IC 10) clusters with confidence-based classifications. We present homogeneous photometry in $I_E$, $Y_E$, $J_E$, $H_E$, and archival UBVRI data, alongside size measurements and properties from BAGPIPES SED fitting. Synthetic cluster injection shows our sample is $\sim 50$% complete to $M \lesssim 10^3 M_\odot$ for ages $\lesssim 100$ Myr, and to $M \lesssim 3\times10^4 M_\odot$ for $\sim 10$ Gyr. IC 10 has more young clusters than NGC 6822, extending to higher masses, consistent with its starburst nature. Both dwarfs host several old massive ($\gtrsim 10^5 M_\odot$) clusters, including an exceptional $1.3 \times 10^6 M_\odot$ cluster in NGC 6822's outskirts. In NGC 6822, we identify a previously undetected, old, extended cluster ($R_h = 12.4 \pm 0.11$ pc). Using well-defined criteria, we identify 11 candidate GCs in NGC 6822 and eight in IC 10. Both galaxies have high specific frequencies ($S_N$) but remain consistent with known GC scaling relations at low luminosity.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.09829-b31b1b.svg)](https://arxiv.org/abs/2509.09829) | **Dynamically New Comet C/2025 D1 (Groeller) with Record Perihelion Distance**  |
|| M.-T. Hui, et al. -- incl., <mark>J. Shi</mark> |
|*Appeared on*| *2025-09-15*|
|*Comments*| *Under review in A&A. Comments are welcome*|
|**Abstract**|            We studied C/2025 D1 (Groeller), a long-period comet with an unprecedented perihelion distance of 14.1 au, using archival observations. The data reveals that it had been active at inbound heliocentric distances $r_{\rm H} \gtrsim 20$ au. Initially, the comet intrinsically brightened at $r_{\rm H} \gtrsim 16$ au, with brightening parameters comparable to those of other long-period comets. However, observations after late 2023 showed a gradual decay, despite the inbound trajectory of the comet. To our knowledge, such behaviours have not been observed for other long-period comets at similar heliocentric distances. We speculate that this might be linked to the onset of CO$_{2}$ sublimation and/or crystallisation processes. The surface brightness profile of the coma indicates a steady-state mass loss, implying supervolatile sublimation as the primary driver of the observed activity. Despite changes in the orbital plane angle, the circularly symmetric coma persisted throughout the observed period, indicative of the dominance of large grains in the coma. Assuming the activity trend is independent of bandpass, we found that comet was redder than many other solar system comets. Our model-dependent constraint estimates the nucleus radius to be $\gtrsim\!0.4$ km. We performed astrometric measurements, refined the orbital solution, and derived the original and future orbits of the comet. Our N-body integration, accounting for the Galactic tide, strongly favours that the comet is dynamically new, with its previous perihelion at $\gtrsim\!60$ au from the Sun $\gtrsim\!6$ Myr ago. It is highly likely that the comet will be lost from our solar system after the current apparition.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.10451-b31b1b.svg)](https://arxiv.org/abs/2509.10451) | **The CHARA Array Polarization Model and Prospects for Spectropolarimetry**  |
|| L. Shuai, et al. -- incl., <mark>S. Kraus</mark> |
|*Appeared on*| *2025-09-15*|
|*Comments*| *37 pages, 24 figures, accepted for publication by AJ. Our code, mircxpol, is published in this https URL*|
|**Abstract**|            Polarimetric data provide key insights into infrared emission mechanisms in the inner disks of YSOs and the details of dust formation around AGB stars. While polarization measurements are well-established in radio interferometry, they remain challenging at visible and near-infrared due to the significant time-variable birefringence introduced by the complex optical beamtrain. In this study, we characterize instrumental polarization effects within the optical path of the CHARA Array, focusing on the H-band MIRC-X and K-band MYSTIC beam combiners. Using Jones matrix formalism, we developed a comprehensive model describing diattenuation and retardance across the array. By applying this model to an unpolarized calibrator, we derived the instrumental parameters for both MIRC-X and MYSTIC. Our results show differential diattenuation consistent with >= 97% reflectivity per aluminum-coated surface at 45 deg incidence. The differential retardance exhibits small wavelength-dependent variations, in some cases larger than we expected. Notably, telescope W2 exhibits a significantly larger phase shift in the Coude path, attributable to a fixed aluminum mirror (M4) used in place of deformable mirrors present on the other telescopes during the observing run. We also identify misalignments in the LiNbO_3 birefringent compensator plates on S1 (MIRC-X) and W2 (MYSTIC). After correcting for night-to-night offsets, we achieve calibration accuracies of $\pm$ 3.4% in visibility ratio and $\pm$ 1.4 deg in differential phase for MIRC-X, and $\pm$ 5.9% and $\pm$ 2.4 deg, respectively, for MYSTIC. Given that the differential intrinsic polarization of spatially resolved sources, such as AGB stars and YSOs, typically greater than these instrumental uncertainties, our results demonstrate that CHARA is now capable of achieving high-accuracy measurements of intrinsic polarization in astrophysical targets.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2509.09762/./canon_cumul_pratio_inv.png', 'tmp_2509.09762/./canon_hist_pratio_inv.png', 'tmp_2509.09762/./psize_kde_m.png', 'tmp_2509.09762/./match_mult.png']
copying  tmp_2509.09762/./canon_cumul_pratio_inv.png to _build/html/
copying  tmp_2509.09762/./canon_hist_pratio_inv.png to _build/html/
copying  tmp_2509.09762/./psize_kde_m.png to _build/html/
copying  tmp_2509.09762/./match_mult.png to _build/html/
exported in  _build/html/2509.09762.md
    + _build/html/tmp_2509.09762/./canon_cumul_pratio_inv.png
    + _build/html/tmp_2509.09762/./canon_hist_pratio_inv.png
    + _build/html/tmp_2509.09762/./psize_kde_m.png
    + _build/html/tmp_2509.09762/./match_mult.png
found figures ['tmp_2509.09878/./YSES1b_Halpha_detail.png', 'tmp_2509.09878/./YSES1b_Hbeta_detail.png', 'tmp_2509.09878/./SGF_with_out_line.png']
copying  tmp_2509.09878/./YSES1b_Halpha_detail.png to _build/html/
copying  tmp_2509.09878/./YSES1b_Hbeta_detail.png to _build/html/
copying  tmp_2509.09878/./S

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\paperone}{\citetalias{2021AAEmsenhuberA}}$
$\newcommand{\papertwo}{\citetalias{2021AAEmsenhuberB}}$
$\newcommand{\mearth}{M_\oplus}$
$\newcommand{\msun}{M_\odot}$
$\newcommand{\mj}{M_{\textrm{\tiny \jupiter}}}$
$\newcommand{\rj}{R_{\textrm{\tiny \jupiter}}}$
$\newcommand{\mstar}{M_\star}$
$\newcommand{\mplanet}{M_\mathrm{planet}}$
$\newcommand{\rplan}{R_\mathrm{plan}}$
$\newcommand{\nobs}{N_{\rm obs}}$</div>



<div id="title">

# The New Generation Planetary Population Synthesis (NGPPS): VII. Statistical comparison with the HARPS/Coralie survey

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.09762-b31b1b.svg)](https://arxiv.org/abs/2509.09762)<mark>Appeared on: 2025-09-15</mark> -  _Accepted for publication in A&A. Abstract abridged to meet arXiv requirements_

</div>
<div id="authors">

A. Emsenhuber, et al. -- incl., <mark>R. Burn</mark>

</div>
<div id="abstract">

**Abstract:** Planetary population synthesis is a tool that is used to better understand the key processes of planet formation at the statistical level. We seek to quantify the fidelity with which modern population syntheses reproduce observations in view of their use as predictive tools. We compared synthetic populations from the Generation 3 Bern Model of Planet Formation and Evolution (core accretion, solar-type host stars) and the HARPS/Coralie radial velocity sample. We biased the synthetic planet population according to the completeness of the observed data. We then performed quantitative statistical comparisons and systematically identified agreements and  differences. Our nominal population reproduces many of the main features of the HARPS planets, such as two main groups of planets in the mass-distance diagram (close-in sub-Neptunes and distant giants), a bimodal mass function with a less populated `desert', an observed mean multiplicity of about 1.6, and several key correlations regarding the stellar metallicity dependency, the period ratio distribution, and the eccentricity distribution. Considering that the model was not optimised beforehand to reproduce any particular survey, this indicates that some of the important physical processes governing planetary formation could be captured. The remaining discrepancies that can be quantified thanks to the population synthesis approach point to areas that are not fully captured in the model. For instance, we find that the synthetic population has 1) in absolute terms too many planets by $\SI{\approx70}{\%}$ , 2) a `desert' that is too deep by $\SI{\approx60}{\%}$ , 3) a relative excess of giant planets by $\SI{\approx40}{\%}$ , 4) planet eccentricities that are on average too low by a factor of about two (median of $\num{0.07}$ versus $\num{0.15}$ ), and 5) a metallicity effect that is too weak. Finally, the synthetic planets are overall too close to the star compared to the HARPS sample.The differences allowed us to find model parameters that better reproduce the observed planet masses, for which we computed additional synthetic populations.We find that decreasing the planet formation efficiency by increasing the planetesimal size re-balances the number of sub-Neptunes versus giant planets. Changing the efficiency of gas-driven migration also affects the sub-Neptune to giant planet ratio, with lower migration rates resulting in more giant planets and fewer sub-Neptunes. However, only modifying the model parameters seems to be insufficient for the model to fully reproduce both the observed mass and distance distributions at the same time. Instead, physical processes appear to be missing. Planets may originate on wider orbits than our model predicts. Mechanisms leading to higher eccentricities and slower disc-limited gas accretion also seem necessary. We also advocate that theoretical models should make a quantitative, rather than merely a qualitative, comparison between the many current and future large surveys and theoretical results to better understand the origins of planetary systems.

</div>

<div id="div_fig1">

<img src="tmp_2509.09762/./canon_cumul_pratio_inv.png" alt="Fig17.1" width="50%"/><img src="tmp_2509.09762/./canon_hist_pratio_inv.png" alt="Fig17.2" width="50%"/>

**Figure 17. -** Cumulative distribution (left) and histogram (right) of the period ratios of adjacent planets for the biased 100-embryo population (\texttt{NG76}, in black), the same population but with _N_-body interactions extended to \SI{100}{\mega\year}(\texttt{NG76longshot}, in blue), and the HARPS/Coralie surveys \citepalias[in red]{2011MayorArxiv}. For the synthetic planets, the values are obtained over \num{1000} simulated observations of the underlying population and showing the \SI{95}{\percent} confidence interval. The number of observable pairs is limited, as only systems with multiple planets are usable. The locations of several MMRs are highlighted. (*fig:comp-pratio*)

</div>
<div id="div_fig2">

<img src="tmp_2509.09762/./psize_kde_m.png" alt="Fig11" width="100%"/>

**Figure 11. -** Comparison of the planet mass distribution between different biased synthetic populations and the observed population. The nominal population with 20 embryos per system (\texttt{NG74}, same as the orange line in Fig. \ref{fig:nemb}) is shown with the black line; the others use the minimum of the Bondi rate and radial flow of the gas disc for the disc-limited gas accretion rate with different planetesimal size, as given in the legend. For each of the synthetic populations, we performed \num{1000} Monte Carlo synthetic observations. The bold line showing the median of these, and the lighter region shows the \SI{95}{\percent} confidence interval. The results are shown as a kernel density estimate, though rescaled to be absolute. (*fig:psize-comp*)

</div>
<div id="div_fig3">

<img src="tmp_2509.09762/./match_mult.png" alt="Fig7" width="100%"/>

**Figure 7. -** Histogram of the number of planets per systems in both the HARPS/Coralie surveys (red) and the optimised synthetic population \texttt{NG192}(green). (*fig:match-mult*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.09762"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\tensorA}{\boldsymbol{\mathcal{A}}}$
$\newcommand{\tensorO}{\boldsymbol{\mathcal{O}}}$
$\newcommand{\tensorPhi}{\boldsymbol{\varPhi}}$
$\newcommand{\tensorD}{\boldsymbol{\mathcal{D}}}$
$\newcommand{\tensorS}{\boldsymbol{\mathcal{S}}}$
$\newcommand{\tensorP}{\boldsymbol{\mathcal{P}}}$
$\newcommand{\tensorE}{\boldsymbol{\mathcal{E}}}$
$\newcommand{\tensorSest}{\boldsymbol{\hat{\mathcal{S}}}}$
$\newcommand{\tensorPest}{\boldsymbol{\hat{\mathcal{P}}}}$
$\newcommand{\Halpha}{\ensuremath{\mathsf{H}_{\upalpha}}\xspace}$
$\newcommand{\Hbeta}{\ensuremath{\mathsf{H}_{\upbeta}}\xspace}$
$\newcommand{\PaBeta}{\ensuremath{\mathsf{Pa}_{\upbeta}}\xspace}$
$\newcommand{\BrGamma}{\ensuremath{\mathsf{Br}_{\upgamma}}\xspace}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\arraystretch}{1.46}$
$\newcommand{\arraystretch}{1.11}$</div>



<div id="title">

# Stellar halo subtraction alternative for accreting companions' characterization with integral field spectroscopy: Analytical and on-sky demonstration on the PDS70, HTLup, and YSES1 systems

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.09878-b31b1b.svg)](https://arxiv.org/abs/2509.09878)<mark>Appeared on: 2025-09-15</mark> -  _14 pages, 15 figures, to be published in A&A_

</div>
<div id="authors">

R. Julo, et al.

</div>
<div id="abstract">

**Abstract:** Medium-resolution integral field spectrographs (IFS), such as the Multi-Unit Spectroscopic Explorer (MUSE) instrument at the Very Large Telescope (VLT), are equipped to detect the emission lines (e.g., $\Halpha, \Hbeta$ ) of faint accreting companions when associated with dedicated stellar halo subtraction methods. We recently proposed a new approach based on polynomial modulations of a stellar spectrum estimate across the field of view, with orthogonal polynomials and lines masking. This new technique is designed to better preserve both continuum and emission lines of accreting companions. We seek to highlight and quantify analytically and on real data the benefits of this new approach over the one classically used, particularly with regard to distortions of the extracted spectra. We also examine both operating regimes. We carried out analytical calculations based on simple toy models of spectra to identify and quantify the main theoretical problems of the state-of-the-art technique, the proposed corrections of our new method, and the remaining limitations of the latter. Simulations of the most extreme situations identified were used to highlight these problems and corrections. Archival VLT/MUSE data of the young PDS70 and HTLup systems were used to vet the detection and characterization capabilities using on-sky observations. New images of the YSES1 planetary system were used to further illustrate the gains. We show that the state-of-the-art stellar halo subtraction method, based on low-pass filtering, can lead to the self-subtraction of the emission lines and modify the neighboring continua, depending on the line contrast to neighboring continuum contrast ratios. We show that the proposed technique corrects these characterization problems, while maintaining the same detection capabilities. The two protoplanets PDS70 b and c were detected with 5 $\sigma$ significance. The $\Halpha$ line estimate of the HTLup B stellar companion was improved by $\sim$ 30 \% for the integrated flux and by $\sim$ 8 \% for the 10 \% -width. As for YSES1 b, we found it uniquely displays a combination of $\Halpha$ , $\Hbeta$ , CaII H \& K triplet, and HeI lines in emission that can be attributed to accretion and/or chromospheric activity. We derived an accretion rate at $\Halpha$ of $\sim1.45 \times 10^{-9}$ $\mathrm{M_{Jup}/year}$ with our new method, versus $\sim1.11 \times 10^{-9}$ $\mathrm{M_{Jup}/year}$ with the reference method, namely, $\sim$ 30 \% less. These new results are compatible with values derived for other companions in this mass range. We note that YSES1 c was not detected in our observations. The proposed subtraction method better preserves the spectral information, notably the emission line fluxes and profiles, while achieving similar detection performance. Based on a linear and parametric approach, it can be extended and/or combined with additional faint signal search algorithms.

</div>

<div id="div_fig1">

<img src="tmp_2509.09878/./YSES1b_Halpha_detail.png" alt="Fig20" width="100%"/>

**Figure 20. -** Zoom in on 51 channels around the $\Halpha$ line of the YSES1 b spectrum estimates, both with the SGF (in blue) and LPM (in orange) methods. Colored areas delimit the 1-$\sigma$ confidence intervals with regard to noise. Dotted Gaussians are fitted to the lines to estimate their main parameters (presented in the legend, with confidence intervals derived from Gaussian fits to the lower and upper bounds of those of the spectrum estimates). The horizontal dots indicate the zero-flux level, i.e., the theoretical flux lower bound. Unlike the LPM method (whose estimate baseline is positive), the SGF method yet cause abnormal negative values (i.e., which cannot be explained by additive electronic noise), as exposed by the Gaussian fits; that is mainly these negative values, unanticipated by the models, that disrupt the usual routines subsequently applied to the estimates. Moreover, even when the Gaussian parameters are calculated relative to their baselines (as is done here) in order to account for this effect as much as possible, it still leads to biased parameter estimations; in fact, self-subtraction not only shifts the lines but also vertically distorts them (as shown in Sect. \ref{sgf}). (*fig:YSES1b_Halpha*)

</div>
<div id="div_fig2">

<img src="tmp_2509.09878/./YSES1b_Hbeta_detail.png" alt="Fig21" width="100%"/>

**Figure 21. -** Same as Fig. \ref{fig:YSES1b_Halpha}, but for the $\Hbeta$ line (the mask used by the LPM method during the estimation process being centered on the $\Hbeta$ line). (*fig:YSES1b_Hbeta*)

</div>
<div id="div_fig3">

<img src="tmp_2509.09878/./SGF_with_out_line.png" alt="Fig2" width="100%"/>

**Figure 2. -** Subtraction results at the position of a planet with the SGF method. Left: Overlapping simulation (ground truth) and estimation of the stellar spaxel. There is no emission line in this simulation. Middle: The green spaxel is the estimation of the planetary spaxel, while the cyan one is the associated ground truth. The self-subtraction problem is clearly visible in this situation. Right: The reason for the self-subtraction is evidenced. This is the deformation estimation, in light pink, overfitting the planetary line component of the data to stellar spectrum estimation ratio, in purple. Yet, this overfitting is relatively greater than the planetary and stellar continuum-to-line ratios are different. The real deformation is in dark pink.
 (*fig:sgf_with_out_line*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.09878"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Cluster Ages to Reconstruct the Milky Way Assembly (CARMA). IV.: Chrono-dynamics of 7 old LMC star clusters and the peculiar origin of NGC 1841

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.10144-b31b1b.svg)](https://arxiv.org/abs/2509.10144)<mark>Appeared on: 2025-09-15</mark> -  _Accepted for publication in A&A, 22 pages, 13 figures, 4 tables_

</div>
<div id="authors">

F. Niederhofer, et al. -- incl., <mark>M. Häberle</mark>

</div>
<div id="abstract">

**Abstract:** In this study, we report conclusive evidence for an ancient star cluster that has been accreted by the Large Magellanic Cloud (LMC). By leveraging observations from the _Hubble_ Space Telescope ( _HST_ ), we investigate the chrono-dynamical structure of a sample of seven old star clusters within the LMC in a self-consistent way. The multi-epoch nature of the dataset allowed the determination of high-precision proper motions for the clusters. Employing an isochrone-fitting methodology, we additionally infer from the deep high-resolution _HST_ data homogeneous and robust estimates for their distances, ages and metallicities. Supplementing these data with literature line-of-sight velocities, we investigate the full 3-dimensional dynamics of the clusters within the frame of the LMC. With respect to the other clusters in our sample, NGC 1841 depicts a peculiar case. Its position in the age-metallicity plane, that makes it about 1 Gyr younger  than the other metal-poor LMC clusters, but also its dynamical properties with a radial orbit almost perpendicular to the LMC disc plane, clearly advocates for a different origin. We thus conclude that NGC 1841 has likely been accreted by the LMC from a smaller galaxy. The other clusters in our sample show disc-like kinematics, with the case of NGC 2210 being peculiar, based on its inclined orbit. Their coherent age-metallicity relation closely resembles that of Gaia-Sausage-Enceladus globular clusters, thus suggesting a similar early evolution for the two dwarf galaxies.  We do not find clear-cut chrono-kinematic evidence that NGC 2005 has been accreted by the LMC as suggested by a previous study based on its chemical abundance pattern. Regardless of its nature, its very old age illustrates that peculiar chemical evolutions already emerge at very early times.

</div>

<div id="div_fig1">

<img src="tmp_2509.10144/./Figs/HODGE11_isofit_ri_i.png" alt="Fig6.1" width="25%"/><img src="tmp_2509.10144/./Figs/HODGE11_isofit_ri_r.png" alt="Fig6.2" width="25%"/><img src="tmp_2509.10144/./Figs/HODGE11_corner_ri_i.png" alt="Fig6.3" width="25%"/><img src="tmp_2509.10144/./Figs/HODGE11_corner_ri_r.png" alt="Fig6.4" width="25%"/>

**Figure 6. -** Isochrone fitting results for Hodge 11. (a) Best-fit isochrone model in the $m_{\rm F814W}$ vs. $m_{\rm F606W}-m_{\rm F814W}$ CMD. (b) Best-fit isochrone model in the $m_{\rm F606W}$ vs. $m_{\rm F606W}-m_{\rm F814W}$ CMD.
        The red X-symbol shown in both CMDs corresponds to the location of the zero-age horizontal branch.
        (c): Corner plot of the posterior probability distributions of pairwise model parameters for the $m_{\rm F814W}$ vs. $m_{\rm F606W}-m_{\rm F814W}$ CMD. The best-fit parameters are quoted in the labels.
        (d): Corner plot of the posterior probability distributions of pairwise model parameters for the $m_{\rm F606W}$ vs. $m_{\rm F606W}-m_{\rm F814W}$ CMD. The best-fit parameters are quoted in the labels.
         (*fig:hodge11_isofit*)

</div>
<div id="div_fig2">

<img src="tmp_2509.10144/./Figs/clusters_ages_compar_rev2.png" alt="Fig4.1" width="25%"/><img src="tmp_2509.10144/./Figs/clusters_mh_compar_rev2.png" alt="Fig4.2" width="25%"/><img src="tmp_2509.10144/./Figs/clusters_dm_compar_rev2.png" alt="Fig4.3" width="25%"/><img src="tmp_2509.10144/./Figs/clusters_ebv_compar_rev2.png" alt="Fig4.4" width="25%"/>

**Figure 4. -** Comparison between the cluster parameters derived in this work and literature values. Top-left panel: Differences between the measured cluster ages and ages derived by [Milone, Cordoni and Marino (2023)](). Top-right panel: Differences between the inferred metallicity and the results from [Grocholski, et. al (2006)](), [Mucciarelli, Massari and Minelli (2021)](), [Song, Mateo and Bailey (2021)]() and [ and Sarajedini (2024)]().
Bottom-left panel: Differences between the distance modulus derived in this study and the measurements from [Wagner-Kaiser, Mackey and Sarajedini (2017)]() and [Milone, Cordoni and Marino (2023)](). Bottom-right panel: Differences between the derived $E(B-V)$ values and the reddenings from [Wagner-Kaiser, Mackey and Sarajedini (2017)]() and [Milone, Cordoni and Marino (2023)](), as well as the resulting values from the reddening maps from [Schlegel, Finkbeiner and Davis (1998)]() and [Skowron, Skowron and Udalski (2021)]().
 (*fig:clusters_compar*)

</div>
<div id="div_fig3">

<img src="tmp_2509.10144/./Figs/clusters_old_v_circ_labels_rev2.png" alt="Fig5.1" width="25%"/><img src="tmp_2509.10144/./Figs/clusters_old_vr_labels_rev2.png" alt="Fig5.2" width="25%"/><img src="tmp_2509.10144/./Figs/clusters_old_vz_labels_rev2.png" alt="Fig5.3" width="25%"/><img src="tmp_2509.10144/./Figs/clusters_old_z_labels_rev2.png" alt="Fig5.4" width="25%"/>

**Figure 5. -** Outline of the clusters' velocities and positions within the LMC. In each panel, NGC 1841 is highlighted with triangle symbol. Panel (a) shows the tangential velocity V$_{\phi}$ as a function of the radial distance from the LMC centre R. Also shown are model circular velocities resulting from a pure NFW profile (orange solid line) and a model composed of a dark matter halo and a stellar disc and bar (green dashed line). The size of the stellar bar is indicated by the vertical dashed line. The radial velocity V$_{R}$ as a function of R is presented in panel (b). Panel (c) illustrates the out-of-plane velocity V$_{Z}$ as a function of R and panel (d) shows the vertical distance from the plane Z as a function of R.
 (*fig:clusters_kinematics*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.10144"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# The large-scale kinematics of young stars in the Milky Way disc: first results from SDSS-V

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.10387-b31b1b.svg)](https://arxiv.org/abs/2509.10387)<mark>Appeared on: 2025-09-15</mark> -  _13 pages (13 figures) + 3 pages of appendices (5 figures). Under review at A&A; revised after first referee comments_

</div>
<div id="authors">

E. Zari, et al. -- incl., <mark>H.-W. Rix</mark>

</div>
<div id="abstract">

**Abstract:** We present a first large-scale kinematic map of $\sim$ 50,000 young OB stars ( $T_{\rm eff} \geq 10,000$ K), based on BOSS spectroscopy from the Milky Way Mapper OB program in the ongoing Sloan Digital Sky Survey V (SDSS-V). Using photogeometric distances, line-of-sight velocities and Gaia DR3 proper motions, we map 3D Galactocentric velocities across the Galactic plane to $\sim$ 5 kpc from the Sun, with a focus on radial motions ( $v_R$ ). Our results reveal mean radial motion with amplitudes of $\pm 30$ km/s that are coherent on kiloparsec scales, alternating between inward and outward motions. These $\bar{v}_R$ amplitudes are considerably higher than those observed for older, red giant populations. These kinematic patterns show only a weak correlation with spiral arm over-densities. Age estimates, derived from MIST isochrones, indicate that 85 \% of the sample is younger than $\sim300$ Myr and that the youngest stars ( $\le 30$ Myr) align well with density enhancements. The age-dependent $\bar{v}_R$ in Auriga makes it plausible that younger stars exhibits different velocity variations than older giants. The origin of the radial velocity features remains uncertain, and may result from a combination of factors,  including spiral arm dynamics, the Galactic bar, resonant interactions, or phase mixing following a perturbation. The present analysis is based on approximately one-third of the full target sample. The completed survey will enable a more comprehensive investigation of these features and a detailed dynamical interpretation.

</div>

<div id="div_fig1">

<img src="tmp_2509.10387/./Figures/density_hot_grayscale.png" alt="Fig12.1" width="50%"/><img src="tmp_2509.10387/./Figures/hot_galcenRV.png" alt="Fig12.2" width="50%"/>

**Figure 12. -** Distribution of the Hot Star Sample in the Galactic plane. The left panel shows the density distribution of the sources. The right panel shows the distribution of sources colour-coded by their Galactocentric radial velocity (computed as described in the text). In both panels, the Sun is in $X,Y =$(-8.2,0) kpc, and it is indicated by the yellow star. The Galactic centre is in $X,Y =$(0,0) kpc. The dashed circles have radii of 2, 4, 6, 8, and 10 kpc. The solid lines indicate constant Galactic longitudes. (*fig:map-galcenrv*)

</div>
<div id="div_fig2">

<img src="tmp_2509.10387/./Figures/overdensity.png" alt="Fig14.1" width="33%"/><img src="tmp_2509.10387/./Figures/vr_overdensities.png" alt="Fig14.2" width="33%"/><img src="tmp_2509.10387/./Figures/vr_spiralarms.png" alt="Fig14.3" width="33%"/>

**Figure 14. -** Comparison between the density distribution of OB stars, the spiral arm model from  \cite{Reid2019}, and the radial velocity asymmetries.  In the left panel and central panels, the solid black contours  represent the  0, and 0.5 density contrast levels. In the right panel, the spiral arms are named following \cite{Reid2019}: Outer arm; Perseus arm; Local arm;  Sagittarius–Carina arm; Scutum–Centaurus arm. In the three panels, the yellow star indicates the Sun's position in $X,Y = -8.2,0$ kpc, and the Galactic centre is located in $X,Y = 0,0$ kpc. The dashed lines represent circle radii of  2, 4, and 6 kpc.  The solid lines indicate constant Galactic longitudes. (*fig:compare_rv*)

</div>
<div id="div_fig3">

<img src="tmp_2509.10387/./Figures/apogee_dr17_vr.png" alt="Fig7" width="100%"/>

**Figure 7. -** Distribution of red giant stars in the Milky Way disc from APOGEE DR17. The red giant stars are colour-coded by their average Galactocentric radial velocity  The location of the Sun is indicated by  the yellow star. The dashed black lines mark concentric circles at $\Delta R=2$ kpc.  The solid lines indicate constant Galactic longitudes. The region within 5 kpc from the Galactic centre is masked as the stellar motions in that region are influenced by the bar. (*fig:apogeeDR17*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.10387"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcid}[1]$
$\newcommand{\arraystretch}{1.4}$</div>



<div id="title">

# $\Euclid$: Early Release Observations -- The star cluster systems of the Local Group dwarf galaxies IC 10 and NGC 6822$\thanks{This paper is published on     behalf of the Euclid Consortium}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.10440-b31b1b.svg)](https://arxiv.org/abs/2509.10440)<mark>Appeared on: 2025-09-15</mark> - 

</div>
<div id="authors">

J. M. Howell, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** Star clusters are valuable indicators of galaxy evolution, offering insights into the buildup of stellar populations across cosmic time. Understanding the intrinsic star cluster populations of dwarf galaxies is particularly important given these systems' role in the hierarchical growth of larger systems.  We use data from $\Euclid$ 's Early Release Observation programme to study star clusters in two star-forming dwarf irregular galaxies in the Local Group, NGC 6822 and IC 10 [ $M_{\star}\sim$ (1--4) $\times10^8 M_{\odot}$ ] . With $\Euclid$ , star clusters are resolved into individual stars across the main bodies and haloes of both galaxies. Through visual inspection of the $\IE$ images, we uncover 30 new star cluster candidates in NGC 6822 and 16 in IC 10, ranging from compact to diffuse extended clusters. We compile and re-evaluate previously identified literature candidates, resulting in final combined catalogues of 52 (NGC 6822) and 71 (IC 10) cluster candidates with confidence-based classifications. We present homogeneous photometry in $\IE$ , $\YE$ , $\JE$ , and $\HE$ , and in archival $_ UBVRI_$ data, alongside size measurements and properties derived from the SED-fitting code \texttt{BAGPIPES} . Through synthetic cluster injection, we conclude our sample is ${\sim}50\%$ complete to $M \lesssim 10^3 {M}_{\odot}$ for ages $\lesssim100 \rm{Myr}$ , and to $M \lesssim 3\times10^4 {M}_{\odot}$ for ages of ${\sim}10 \rm{Gyr}$ . We find that IC 10 has more young clusters than NGC 6822, and its young clusters extend to higher masses, consistent with its starburst nature.  We find several old massive ( $\gtrsim10^5 M_{\odot}$ ) clusters in both dwarfs, including an exceptional cluster in NGC 6822's outskirts with a mass of $1.3 \times 10^6 M_{\odot}$ , nearly twice as massive as any other old cluster in either galaxy.  In NGC 6822, we also identify a previously undetected, old, and extended cluster ( $R_{\mathrm{h}} = 12.4 \pm 0.11 \mathrm{pc}$ ). Using well-defined criteria, we identify 11 candidate GCs in NGC 6822 and eight in IC 10.  Both galaxies have high specific frequencies ( $S_{\rm N}$ ) for their luminosities but remain consistent with the known GC scaling relationships in the low-luminosity regime.

</div>

<div id="div_fig1">

<img src="tmp_2509.10440/./full_and_zoom_6822_annotated_high_res_compressed.png" alt="Fig14" width="100%"/>

**Figure 14. -** Examples of old and young clusters across NGC 6822, both previously found and new from this study, showcased alongside an RGB image of the \Euclid data displaying its full FoV. \IE, \YE, and \HE are the blue, green, and red channels, respectively. Each cutout is ${\sim}44$ pc on the side in the \IE band, displayed in log scale. The zoom-in region is roughly $16$\farcm$5 \times 13$\farcm$5$($2.5 \text{kpc} \times 2  \text{kpc}$). The top row includes the GCs SC7 (ESCC-NGC6822-08) and SC5 (ESCC-NGC6822-06), the middle row shows the cluster Hubble-VI (ESCC-NGC6822-01), and the bottom row features a newly identified extended cluster candidate, ESCC-NGC6822-27. (*fig:example_clusts*)

</div>
<div id="div_fig2">

<img src="tmp_2509.10440/./Agevsmass.png" alt="Fig19" width="100%"/>

**Figure 19. -** Masses as a function of age as derived by \texttt{BAGPIPES} for the star clusters in NGC 6822 (left) and IC 10 (right). The blue line shows the 50\% completeness limit in the inner regions. Also plotted in grey is the sample of SMC clusters from \citet{Gatto2021}. (*fig:age_vs_mass*)

</div>
<div id="div_fig3">

<img src="tmp_2509.10440/./Rh_vs_Vmag.png" alt="Fig7" width="100%"/>

**Figure 7. -** Half-light radii as a function of absolute _ V_-band magnitude for the old cluster sample, and for the sample of GCs from \citep{Georgiev2009}. The empirical relation  $\logten(R_{\rm h}/{\rm pc}) = 0.25  {M_V} + 2.95$\citep{Mackey2005}, which defines the upper envelope of MW GCs, is overplotted. Small red circles mark the three most extended clusters in NGC 6822.   (*fig:Rh_vs_Vmag*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.10440"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

130  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

11  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

5  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
